In [1]:
import Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate()

  Activating environment at `~/tinympc-julia/state_constraints/Project.toml`


In [2]:
using RobotZoo:Quadrotor
using RobotDynamics
using ForwardDiff
using TrajOptPlots
using BlockDiagonals
using LinearAlgebra
using StaticArrays
using SparseArrays

using MeshCat
using ColorTypes
using GeometryBasics: HyperRectangle, Cylinder, Vec, Point, Mesh
using CoordinateTransformations
using Rotations

using Plots
using Printf


include("../quaternion-stuff.jl")

E (generic function with 1 method)

In [3]:
# Quadrotor parameters
m = 0.035
ℓ = 0.046/sqrt(2)
J = [16.6e-6 0.83e-6 0.72e-6;
    0.83e-6 16.6e-6 1.8e-6;
    0.72e-6 1.8e-6 29.3e-6];
gravity = 9.81
thrustToTorque = 0.0008
scale = 65535
kt = 2.245365e-6*scale # u is PWM in range [0...1]
km = kt*thrustToTorque #4.4733e-8

# From "design of a trajectory tracking controller for a nanoquadcopter"
# J = [1.395e-5 0 0;
#     0 1.436e-5 0;
#     0 0 2.173e-5];
# kt = .2025
# km = .11

# h = 1/8000
# h = 1/5000
# h = 1/1000
# h = 1/500
# h = 1/250
# h = 1/100
h = 1/50
# h = 1/25
# h = 1/10

Nx = 13     # number of states (quaternion)
Nx̃ = 12     # number of states (linearized)
Nu = 4      # number of controls
nx = Nx̃
nu = Nu

#Goal state
uhover = (m*gravity/kt/4)*ones(4) # m = 30g and max thrust = 60g
# uhover = [0.701, 0.701, 0.832, 0.732]
rg = [0.0; 0; 0.0]
qg = [1.0; 0; 0; 0]
vg = zeros(3)
ωg = zeros(3)
xg = [rg; qg; vg; ωg];

In [4]:
function quad_dynamics(x,u)
  r = x[1:3]
  q = x[4:7]/norm(x[4:7]) #normalize q just to be careful
  v = x[8:10]
  ω = x[11:13]
  Q = qtoQ(q)
  
  ṙ = v
  q̇ = 0.5*L(q)*H*ω
  
  b = 1/m

  v̇ = [0; 0; -gravity] + (1/m)*Q*[zeros(2,4); kt*ones(1,4)]*u 

  ω̇ = J\(-hat(ω)*J*ω + [-ℓ*kt -ℓ*kt ℓ*kt ℓ*kt; -ℓ*kt ℓ*kt ℓ*kt -ℓ*kt; -km km -km km]*u)
  
  return [ṙ; q̇; v̇; ω̇]
end
function quad_dynamics_rk4(x,u)
  #RK4 integration with zero-order hold on u
  f1 = quad_dynamics(x, u)
  f2 = quad_dynamics(x + 0.5*h*f1, u)
  f3 = quad_dynamics(x + 0.5*h*f2, u)
  f4 = quad_dynamics(x + h*f3, u)
  xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
  xn[4:7] = xn[4:7]/norm(xn[4:7]) #re-normalize quaternion
  return xn
end

quad_dynamics_rk4 (generic function with 1 method)

In [5]:
#Linearize dynamics about hover
Adyn = ForwardDiff.jacobian(x->quad_dynamics_rk4(x,uhover),xg)
Bdyn = ForwardDiff.jacobian(u->quad_dynamics_rk4(xg,u),uhover);
Ãdyn = Array(E(qg)'*Adyn*E(qg))
B̃dyn = Array(E(qg)'*Bdyn);

In [6]:
vis = Visualizer()
quad_model = Quadrotor()
TrajOptPlots.set_mesh!(vis, quad_model)

function TransformPlane(visObject, center, dir)

    tol = 1e-4
    x_axis = [1.0; 0.0; 0.0]

    # Compute axis angle
    dot = x_axis'*dir
    if dot > 1-tol
        axis = x_axis
    elseif dot < -1+tol
        axis = -x_axis
    else
        axis = cross(x_axis, dir)
    end
    angle = acos(x_axis'*dir)

    settransform!(visObject, Translation(center...) ∘ LinearMap(AngleAxis(angle, axis...)))
end

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8703
└ @ MeshCat /home/tinympc/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


TransformPlane (generic function with 1 method)

In [7]:
# Cost weights
max_dev_x = [0.01; 0.01; 0.01; 0.5; 0.5; 0.05;  0.5; 0.5; 0.5;  0.7; 0.7; 0.5]
max_dev_u = [0.1; 0.1; 0.1; 0.1]
# max_dev_u = [0.02; 0.02; 0.02; 0.02]
Q = diagm(1 ./(max_dev_x.^2))
R = diagm(1 ./(max_dev_u.^2))
# Q = spdiagm([100, 100, 100,  4, 4, 400,  4, 4, 4,  2, 2, 4])
# R = spdiagm([4, 4, 4, 4])
# Q = spdiagm([100, 100, 400,  4, 4, 100,  4, 4, 4,  2, 2, 4]) #.*5^2
# R = spdiagm([5, 5, 5, 5]) #.*5^2
Qf = Q
# ρ = 5
# R̃ = R + ρ*I;
# Q̃ = Q + ρ*I;
# Q̃f = Qf + ρ*I;
;

In [8]:
# Number of backward pass related matrix sets to cache
ρ_list_length = 1

# Allocate cache
cache = (
    A = Ãdyn,
    B = B̃dyn,
    # ρ_list = [diagm(ones(nu)) for _ in 1:ρ_list_length],
    ρ_list = [zeros(1) for _ in 1:ρ_list_length],
    Kinf_list = [zeros(nu,nx) for _ in 1:ρ_list_length],
    Pinf_list = [zeros(nx,nx) for _ in 1:ρ_list_length],
    Quu_inv_list = [zeros(nu,nu) for _ in 1:ρ_list_length],
    AmBKt_list = [zeros(nx,nx) for _ in 1:ρ_list_length],
    coeff_d2p_list = [zeros(nx,nu) for _ in 1:ρ_list_length],
)

# Precompute all cached matrices for multiple ρ values starting at ρ = .1 and multiplying by 5
for k = 1:ρ_list_length
    # ρ
    # ρ = diagm([1; 1; 1; 1])*1*5^(k-1)
    ρ = 5*(5^(k-1));
    R̃ = R + ρ*I;
    Q̃ = Q + ρ*I;
    Q̃f = Qf + ρ*I;

    # K1 = dlqr(cache.A, cache.B, Q̃, R̃)
    # display(K)
    
    K = zeros(nu,nx)
    P = zeros(nx,nx)
    Kprev = zeros(nu,nx)
    Pprev = zeros(nx,nx)
    
    # Compute Kinf, Pinf
    riccati_iters = 0
    riccati_err = 1e-10
    Pprev = Q̃f
    while true
        K = (R̃ + cache.B'*Pprev*cache.B)\(cache.B'*Pprev*cache.A);
        P = Q̃ + cache.A'*Pprev*(cache.A - cache.B*K);
        if maximum(abs.(K - Kprev)) < riccati_err
            break
        end
        Kprev = K
        Pprev = P
        riccati_iters += 1
    end
    display(K)
    # display(maximum(K1-K))

    # Cache precomputed values
    cache.ρ_list[k] .= ρ;
    # cache.Kinf_list[k] .= K[1];
    # cache.Pinf_list[k] .= P[1];
    cache.Kinf_list[k] .= K;
    cache.Pinf_list[k] .= P;
    cache.Quu_inv_list[k] .= (R̃ + cache.B'*cache.Pinf_list[k]*cache.B)\I;
    cache.AmBKt_list[k] .= (cache.A - cache.B*cache.Kinf_list[k])';
    cache.coeff_d2p_list[k] .= cache.Kinf_list[k]'*R̃ - cache.AmBKt_list[k]*cache.Pinf_list[k]*cache.B;
end

4×12 Matrix{Float64}:
 -1.33252   1.33521  4.28413  -1.23061  …  -0.0546082  -0.0647876  -0.279677
  1.23343   1.22886  4.28413  -1.05931     -0.0369556   0.0609616   0.279334
  1.31463  -1.32819  4.28413   1.15196      0.040783    0.0302132  -0.278478
 -1.21554  -1.23588  4.28413   1.13796      0.0507808  -0.0263872   0.278821

In [9]:
# Create reference sketch trajectory
Tfinal = 5
N = Int(Tfinal/h)+1

X̃ref = [zeros(nx) for i = 1:N]
# Xref = [[0; 0; 1; 1; zeros(9)] for i = 1:N] # Point
Xref = [[0; t; 1; 1; zeros(9)] for t in range(-1.5, 1.5, length=N)] # Line
# Xref = [[sin(t); cos(t); 1; 1; zeros(9)] for t in range(-pi/2, 3*pi/2, length=N)] # Circle
# Xref = [[sin(2*t); 2*cos(t); 1; 1; zeros(9)] for t in range(-pi/2, 3*pi/2, length=N)] # Figure-eight
Uref = [zeros(nu) for i = 1:N-1]

# Compute reference velocity from reference position
for i = 1:N-1
    Xref[i][8:10] = (Xref[i+1][1:3] - Xref[i][1:3])/h
end

# Convert (13 state) Xref to reduced form (12 state) X̃ref
for k = 1:N
    x = Xref[k]
    qx = x[4:7]
    ϕ = qtorp(L(qg)'*qx)   
    X̃ref[k] = [x[1:3]; ϕ; x[8:10]; x[11:13]]
end

# Set initial state
x0 = X̃ref[1] + [0; 0; 0; zeros(9)]
# x0 = [0.1; 0.1; 0; zeros(9)]

# Visualize reference
delete!(vis["XrefLine"])
XrefLine = [Point(x_[1], x_[2], x_[3]) for x_ in Xref]
setobject!(vis["XrefLine"], Object(PointCloud(XrefLine), 
        LineBasicMaterial(color=Colors.RGBA(0.,0.45,1.)), "Line"))
;

In [10]:
# Initialize normal distribution to add noise to the estimate of x
using Random, Distributions
Random.seed!(123)

ArgumentError: ArgumentError: Package Distributions not found in current path:
- Run `import Pkg; Pkg.add("Distributions")` to install the Distributions package.


In [11]:
using Libdl
# lib = dlopen("/Users/anoushkaalavill/Documents/REx_Lab/TinyMPC/lib_test") # library pointer
# func = dlsym(lib, "my_add") # function pointer
# function call_c(x, y)
#     @ccall $func(x::Cint, y::Cint)::Cint
# end
# output = call_c(5, 5)
# @show output

lib = dlopen("/home/tinympc/tinympc-crazyflie-firmware/examples/controller_tinympc_eigen_task/TinyMPC/build/src/tinympc/libtinympc.so") # library pointer
func_admm = dlsym(lib, "julia_sim_wrapper_solve_admm") # function pointer
function call_c_admm(x, u, mpc_iter, x_max, A_ineq)
    # @ccall $func(x::Ref{Cfloat}, y::Ref{Cfloat})::Cvoid # C takes 32-bit floats
    @ccall $func_admm(x::Ref{Cfloat}, u::Ref{Cfloat}, mpc_iter::Int, x_max::Ref{Cfloat}, A_ineq::Ref{Cfloat})::Cvoid # C takes 32-bit floats
end

func_lqr = dlsym(lib, "julia_sim_wrapper_solve_lqr") # function pointer
function call_c_lqr(x, u)
    # @ccall $func(x::Ref{Cfloat}, y::Ref{Cfloat})::Cvoid # C takes 32-bit floats
    @ccall $func_lqr(x::Ref{Cfloat}, u::Ref{Cfloat})::Cvoid # C takes 32-bit floats
end

call_c_lqr (generic function with 1 method)

In [15]:
include("../tinyMPC-ADMM-dt-state.jl")

speye(N) = spdiagm(ones(N))


# Clean up MeshCat environment from previous run
for k in 1:10000
    delete!(vis["xlin " * string(k)])
    delete!(vis["linearized constraint " * string(k)])
    delete!(vis["obstacle " * string(k)])
    delete!(vis["bounding obstacle " * string(k)])
end



# Used for skipping skip_obs number of linearized obstacle constraints (skips both visualization and computation)
skip_obs = 1

# MPC setup
Nh = 25


# Create obstacle in MeshCat
obs_center = [0.0; 0.0; 0.8]

r_vis = 0.1
r_obstacle = 0.5


# Create visualization data to populate during MPC solution

bounding_material = MeshPhongMaterial(color=RGBA(1, 1, 1, 0.25))
thin_rect_material = MeshPhongMaterial(color=RGBA(0, 1, 0, 0.25))
constraintPlanes = [setobject!(vis["linearized constraint " * string(k)], HyperRectangle(Vec(-0.0005,-0.5,-0.5), Vec(0.001,1,1)), thin_rect_material) for k in 1:skip_obs:Nh]

# Visualize sphere
setobject!(vis["obstacle"], HyperSphere(Point(obs_center...), r_vis))
setobject!(vis["bounding obstacle"], HyperSphere(Point(obs_center...), r_obstacle), bounding_material)

# Define number of state constraints (nm)
nm = 1

# Instantiate TinyMPC-ADMM variables

# x = [zeros(nx) for i = 1:Nh]
# u = [zeros(nu) for i = 1:Nh-1]
x = zeros(Float32, nx, Nh)
u = zeros(Float32, nu, Nh-1)

q = [zeros(nx) for i = 1:Nh];
r = [zeros(nu) for i = 1:Nh-1];

p = [zeros(nx) for i = 1:Nh];      # cost to go linear term
p[Nh] = q[Nh];

d = [zeros(nu) for i = 1:Nh-1];    # feedforward control

# Input constraints
umin = -[1.0; 1.0; 1.0; 1.0]*.5
umax = [1.0; 1.0; 1.0; 1.0]*.5

# State constraints
# xmin = [-Inf .* ones(nx) for i = 1:Nh]
# xmax = [[Inf; 0.5; Inf .* ones(10)] for i = 1:Nh]
xmin = [-Inf for i = 1:Nh] # Only one -Inf per knot point because only one inequality constraint (for now)
# xmax = [zeros(2) for i = 1:Nh]
# xmax = [Inf*ones(1) for i = 1:Nh]
xmax = zeros(Float32, Nh)

# Aineq = [speye(nx) for i = 1:Nh]
# Aineq = [zeros(nx) for i = 1:Nh]
Aineq = zeros(Float32, nx, Nh)

for k = 1:skip_obs:Nh
    xc = obs_center - X̃ref[k][1:3]
    a = xc/norm(xc)

    # Aineq[k][1:3] = a'
    Aineq[1:3, k] = a

    q_c = obs_center - r_obstacle*a
    b = a'*q_c
    # xmax[k][1] = b
    xmax[k] = b
    TransformPlane(constraintPlanes[cld(k,skip_obs)], q_c, a)
end


# Simulate
Nsim = N - Nh
xhist = [zeros(nx+1) for _ = 1:Nsim] # nx+1 because not reduced form
xhist[1] = [x0[1:3]; rptoq(x0[4:6]); x0[7:12]]
uhist = [zeros(nu) for _ = 1:Nsim-1]

prev_Δu = 0
for k = 1:Nsim-1 # Nsim-1 knotpoints for control, Nsim knotpoints for state

    # State slack (auxiliary) and dual variables
    v = [zeros(nx) for i = 1:Nh]
    vnew = [zeros(nx) for i = 1:Nh]
    g = [zeros(nx) for i = 1:Nh]

    # Input slack (auxiliary) and dual variables
    z = [zeros(nu) for i = 1:Nh-1]
    znew = [zeros(nu) for i = 1:Nh-1]
    y = [zeros(nu) for i = 1:Nh-1]

    # x[1] = [xhist[k][1:3]; qtorp(xhist[k][4:7]); xhist[k][8:end]]
    # d = Normal()
    # noise = 0.01*rand(d, 12)
    x[:,1] = [xhist[k][1:3]; qtorp(xhist[k][4:7]); xhist[k][8:end]]# + noise # add noise to estimate of x

    params = (
        N = Nh,
        Q = Q, R = R, Qf = cache.Pinf_list[1],
        A = Aineq,
        umin = umin, umax = umax,
        xmin = xmin, xmax = xmax,
        Xref = X̃ref[k:k+Nh-1], Uref = Uref[k:k+Nh-2], 
        cache = cache,
        uhover = uhover
    );

    # Solve
    # Δu, status, iter = solve_admm!(vis, params, q, r, p, d, x,v,vnew,g, u,z,znew,y; ρ=ρ, abs_tol=1e-2, max_iter=10000)
    # u = -cache.Kinf*(x[1] - X̃ref[1]) 
    display("new iter\n\n")
    call_c_admm(x, u, k, xmax, Aineq)

    # display(x)
    
    # Visualize solution x
    delete!(vis["xLinearized"])
    x_linearized = [Point(x[:,i][1], x[:,i][2], x[:,i][3]) for i in 1:Nh]
    setobject!(vis["xLinearized"], Object(PointCloud(x_linearized), 
    LineBasicMaterial(color=Colors.RGBA(1,0.6,0.0)), "Line"))

    Δu = u
    # Roll out x_k+1 with actual dynamics
    # if prev_Δu == 0
    #     prev_Δu = Δu[:,1]
    # end
    uhist[k] = u[:,1] + uhover
    # uhist[k] = Δu[1] + uhover
    # uhist[k] = u_lqr + uhover

    xhist[k+1] = quad_dynamics_rk4(xhist[k], uhist[k])
        
    # # Roll out with real dynamics
    # x_real = [zeros(nx+1) for _ = 1:Nh]
    # x_real[1] = xhist[k]
    # for j = 1:Nh-1
    #     x_real[j+1] = quad_dynamics_rk4(x_real[j], uhist[k])
    # end

    for j = 1:skip_obs:Nh
        xc = obs_center - x[1:3, j]
        a = xc/norm(xc) # hyperplane projection expects normalized a

        # Aineq[k][1:3] = a'
        Aineq[1:3, j] = a

        q_c = obs_center - r_obstacle*a
        b = a'*q_c
        # xmax[j][1] = b
        xmax[j] = b
        dist = a'*x[1:3, j] - b
        # if dist > 0
        #     display(dist)
        # end
        TransformPlane(constraintPlanes[cld(j,skip_obs)], q_c, a)
    end
end

delete!(vis["xHistline"])
xHistline = [Point(x_[1], x_[2], x_[3]) for x_ in xhist]
setobject!(vis["xHistline"], Object(PointCloud(xHistline), 
LineBasicMaterial(color=Colors.RGBA(0.75,0.5,0.8)), "Line"))
     

"new iter\n\n"

"new iter\n\n"

"new iter\n\n"

"new iter\n\n"

"new iter\n\n"

...

"new iter\n\n"

"new iter\n\n"

"new iter\n\n"

prev xyz: -0.000790243
    0.126844
   -0.038724
vnew: -0.000993126
    0.613637
   0.0516186
prev xyz: -0.000797418
    0.127936
  -0.0387232
vnew: -0.0010008
  0.612808
 0.0503447
prev xyz: -0.000804605
    0.129028
  -0.0387224
vnew: -0.0010084
   0.61199
 0.0490982
prev xyz: -0.000811793
    0.130119
  -0.0387216
vnew: -0.00101591
   0.611182
  0.0478785
prev xyz: -0.000819098
    0.131211
  -0.0387208
vnew: -0.00102345
   0.610384
  0.0466847
prev xyz: -0.00082639
   0.132302
   -0.03872
vnew: -0.00103091
   0.609596
  0.0455161
prev xyz: -0.000833698
    0.133393
  -0.0387191
vnew: -0.00103831
   0.608818
  0.0443719
prev xyz: -0.000841076
    0.134485
  -0.0387183
vnew: -0.00104571
   0.608049
  0.0432513


MeshCat Visualizer with path /meshcat/xHistline at http://127.0.0.1:8703

In [13]:
X1 = [SVector{13}(x_) for x_ in xhist];
visualize!(vis, quad_model, Tfinal, X1)